## Конвертация естественного языка в запросы SQL

В грубом приближении этапы работы RAG через SQL заключаются в следующих пунктах:

- `Привести вопрос пользователя на естественном языке к запросу SQL`
- `Исполнить запрос`
- `Дать модели результат и сгенерировать ответ пользователю с учётом полученных данных`

In [1]:
from langchain.utilities import SQLDatabase
from langchain_community.llms import YandexGPT
from langchain_community.chat_models import ChatYandexGPT
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import create_sql_query_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import sqlite3

from langchain_core.globals import set_debug
#set_debug(True)

In [ ]:
from langchain.chains.sql_database.prompt import PROMPT
print(PROMPT.template)

In [2]:
load_dotenv()

True

In [ ]:
import pandas as pd

database_name = "analytics.db"
csv_files = {
    "customer":"data/customer.csv",
    "employee":"data/employee.csv",
    "sale":"data/sale.csv"
}

conn = sqlite3.connect(database_name)

for table_name, csv_path in csv_files.items():
    df = pd.read_csv(csv_path)
    df.to_sql(
        name=table_name,
        con=conn,
        if_exists="replace",
        index=False
    )

conn.close()

In [3]:
llm = ChatYandexGPT(model_name="yandexgpt", temperature=0)

In [6]:
db = SQLDatabase.from_uri("sqlite:///analytics.db", sample_rows_in_table_info=1)

template = '''Ты эксперт по SQLite. Получив вопрос после Question, сперва сгенерируй синтаксически верный SQLite запрос, отвечающий на этот вопрос, и запиши его, затем просмотри результат запроса и верни ответ на вопрос.
Запрашивай не более чем {top_k} результатов за запрос
Используй следующий формат:

Question: "Вопрос здесь"
SQLQuery: "SQL запрос для выполнения"
SQLResult: "Результат SQL запроса"
Answer: "Конечный ответ здесь"

Используй только следующие имена таблиц:

{table_info}.

Question: {input}'''

prompt = PromptTemplate.from_template(template)

chain = create_sql_query_chain(llm, db,)

print(chain.middle[1])

input_variables=['input', 'table_info'] input_types={} partial_variables={'top_k': '5'} template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use date(\'now\') function to get the current date, if the question invo

In [5]:
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT count(*) FROM customer LIMIT 10;")

sqlite
['customer', 'employee', 'sale']


'[(1000,)]'

In [7]:
response = chain.invoke({"question": "Сколько у нас работников?"})
response

'```\nSELECT COUNT(*) FROM employee;\n```'

In [11]:
response.strip("`\n")

'SELECT COUNT(*) FROM employee;'

In [12]:
db.run(response.strip("`\n"))

'[(11,)]'

In [ ]:
response = chain.invoke({"question": "Сколько клиентов из Бронкса?"})
print(response)
print(db.run(response.strip("`\n")))

In [16]:
response = chain.invoke({"question": "Сколько клиентов из Бронкса (используй английское название)?"})
print(response)
print(db.run(response.strip("`\n")))

```
SELECT COUNT(customerid) 
FROM customer 
WHERE city = "Bronx";
```
[(2,)]


In [14]:
response = chain.invoke({"question": "Из какого штата наибольшее количество клиентов?"})
print(response)
print(db.run(response.strip("`\n")))

```
SELECT state, COUNT(*) AS count 
FROM customer 
GROUP BY state 
ORDER BY count DESC 
LIMIT 1;
```
[('NY', 79)]


In [ ]:
response = chain.invoke({"question": "Найди мне топ 5 клиентов, заплативших наибольшую стоимость пересылки"})
print(response)
print(db.run(response.strip("`\n")))